<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelsDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
import numpy as np

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [58]:
# Importiamo il dataset ottenuto tramite le operazioni di Data Preprocessing
dataset = pd.read_csv("tennis.csv")

for col in dataset.columns:
    print(col)

Unnamed: 0
match_id
Pt
Set1
Set2
Gm1
Gm2
Pts
Player1
Player2
Ranking1
Ranking2
Svr
PtWinner
PointType
SetID
PtSet
SetWinner
MatchWinner
p1_win_nobreak_point
p2_win_nobreak_point
p1_win_break_point
p2_win_break_point
p1_lost_nobreak_point
p2_lost_nobreak_point
p1_lost_break_point
p2_lost_break_point
serve_ace_1
serve_ace_2
serve_miss2_1
serve_miss1_2
rally_winner_1
rally_winner_2
rally_forced2_1
rally_forced1_2
rally_unforced1_2
rally_unforced2_1


#**Train-Test split**

In [43]:
# Estrai tutti gli ID unici delle partite
match_ids = dataset['match_id'].unique()

# Split degli ID in training e test (es. 80% train, 20% test)
train_ids, test_ids = train_test_split(match_ids, test_size=0.2, random_state=42)

# Filtra il dataset mantenendo intere le partite
train_data = dataset[dataset['match_id'].isin(train_ids)].reset_index(drop=True)
test_data = dataset[dataset['match_id'].isin(test_ids)].reset_index(drop=True)

Adesso aumento la dimensionalità degli split considerando le istanze simmetriche

In [44]:
def augment_with_symmetric(df: pd.DataFrame) -> pd.DataFrame:
    df_swapped = df.copy()

    # Colonne da swappare a coppie
    swap_pairs = [
        ('Set1', 'Set2'),
        ('Gm1', 'Gm2'),
        ('Player1', 'Player2'),
        ('Ranking1', 'Ranking2'),
        ('p1_win_nobreak_point', 'p2_win_nobreak_point'),
        ('p1_win_break_point', 'p2_win_break_point'),
        ('p1_lost_nobreak_point', 'p2_lost_nobreak_point'),
        ('p1_lost_break_point', 'p2_lost_break_point'),
        ('serve_ace_1', 'serve_ace_2'),
        ('serve_miss2_1', 'serve_miss1_2'),
        ('rally_winner_1', 'rally_winner_2'),
        ('rally_forced2_1', 'rally_forced1_2'),
        ('rally_unforced1_2', 'rally_unforced2_1')
    ]
    for col1, col2 in swap_pairs:
        df_swapped[[col1, col2]] = df[[col2, col1]].values

    # Colonne dove 1 <-> 2
    invert_1_2_cols = ['Svr', 'PtWinner', 'SetWinner', 'MatchWinner']
    for col in invert_1_2_cols:
        df_swapped[col] = df[col].replace({1: 2, 2: 1})

    # Inversione della colonna Pts (es. "40-15" -> "15-40")
    def invert_pts(value):
        try:
            left, right = value.split('-')
            return f"{right}-{left}"
        except:
            return value  # Se non è nel formato previsto, lo lascia invariato

    if 'Pts' in df.columns:
        df_swapped['Pts'] = df['Pts'].apply(invert_pts)

    # Aggiunta del suffisso 'simm' a match_id e set_id, se presenti
    for col in ['match_id', 'set_id']:
        if col in df_swapped.columns:
            df_swapped[col] = df_swapped[col].astype(str) + '_simm'

    return pd.concat([df, df_swapped], ignore_index=True)


train_data = augment_with_symmetric(train_data)
test_data = augment_with_symmetric(test_data)

Bisogna convertire i valori non numerici in valori numerici e poi bisogna convertire i valori numerici in valori adatti all'addestramento

Creare i batch di training del Modello 1(il migliore su Github): Quindi sono con i ranking e chi vince

Usare la predizione di partenza di tale modello per:

Modello 2 LSTM: per ogni match id considerare tutte le istanze di punteggi con le colonne utili con etichetta match winner
Provare variante sui set quindi dividere per set id e mettere etichetta SetWinner

Crea la Loss Function corretta dando peso alle giuste feature e ai punti consecutivi


Considera di usare la predizione del modello 1 con la percentuale data dalla funzione matematica


Eventualmente fare test su determinati Player

Togliere poi le colonne inutili

In [59]:
dataset = dataset.drop(columns=['PointType','Player1','Player2'])
dataset

,Unnamed: 0,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Ranking1,Ranking2,...,serve_ace_1,serve_ace_2,serve_miss2_1,serve_miss1_2,rally_winner_1,rally_winner_2,rally_forced2_1,rally_forced1_2,rally_unforced1_2,rally_unforced2_1
0,0,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,1,0,0,0,0,0-0,109,2,...,0,0,0,0,0,0,0,0,0,1
1,1,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,2,0,0,0,0,0-15,109,2,...,0,0,0,0,0,0,0,1,0,0
2,2,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,3,0,0,0,0,0-30,109,2,...,1,0,0,0,0,0,0,0,0,0
3,3,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,4,0,0,0,0,15-30,109,2,...,0,0,0,0,0,0,0,0,0,1
4,4,20241229-M-United_Cup-RR-Thiago_Monteiro-Alexa...,5,0,0,0,0,15-40,109,2,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988487,1022099,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,247,2,2,4,3,40-0,11,7,...,0,0,0,0,0,0,0,0,0,0
988488,1022100,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,248,2,2,5,3,0-0,11,7,...,0,0,0,0,0,0,0,0,0,0
988489,1022101,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,249,2,2,5,3,15-0,11,7,...,0,0,0,0,0,0,0,0,0,0
988490,1022102,19600529-M-Roland_Garros-F-Nicola_Pietrangeli-...,250,2,2,5,3,30-0,11,7,...,0,0,0,0,0,0,0,0,0,0


#**Model 1**

# **Model 2**